In [1]:
import pandas as pd
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
lemmer = PorterStemmer()
stop_words_nltk = set(stopwords.words('english'))

In [22]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [18]:
# nltk.download('omw-1.4')
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ifte-
[nltk_data]     home/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
df = pd.read_csv('ROCStories_winter2017 - ROCStories_winter2017.csv')

df

,storyid,storytitle,sentence1,sentence2,sentence3,sentence4,sentence5
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David Drops the Weight,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Frustration,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Different Opinions,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...
4,c71bb23b-7731-4233-8298-76ba6886cee1,Overcoming shortcomings,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.
...,...,...,...,...,...,...,...
52660,134e8636-3617-43d8-ba6a-9a11b3b115b1,Flavor,The man liked the flavor.,He tried to recreate it at home.,He could not get the flavor right.,He asked the owner of the recipe for help.,The owner of the flavor sold him the recipe.
52661,4c317f76-ca42-4024-a4c2-12ec911cf89b,After Death,"After my friend's dad's funeral, I got in trou...",The principal said I wasn't allowed to leave s...,He found out I had my friend sign me out.,He told me I was getting detention.,I skipped detention all week.
52662,a18fd0d2-4d0c-4316-befe-e3d827fe699b,Janice breaks her wrist,Janice was out exercising for her big soccer g...,She was doing some drills with her legs.,While working out and exercising she slips on ...,She falls down and uses her wrist to break her...,She breaks her wrist in the process and goes t...
52663,2c14252b-4080-4fca-8765-537772018508,Jamie marries for love,Jamie is an american girl.,Jamie wants to get married to a mexican man.,Her family assumes it's because the man wants ...,Jamie insist that she is marrying him out of l...,Jamie gets married and they spent the rest of ...


In [22]:
df['story'] = df['sentence1'] + df['sentence2'] + df['sentence3'] + df['sentence4'] + df['sentence5']

In [23]:
df = df[['storytitle', 'story']]

In [7]:
df

,storytitle,story
0,David Drops the Weight,David noticed he had put on a lot of weight re...
1,Frustration,Tom had a very short temper.One day a guest ma...
2,Marcus Buys Khakis,Marcus needed clothing for a business casual e...
3,Different Opinions,Bobby thought Bill should buy a trailer and ha...
4,Overcoming shortcomings,John was a pastor with a very bad memory.He tr...
...,...,...
52660,Flavor,The man liked the flavor.He tried to recreate ...
52661,After Death,"After my friend's dad's funeral, I got in trou..."
52662,Janice breaks her wrist,Janice was out exercising for her big soccer g...
52663,Jamie marries for love,Jamie is an american girl.Jamie wants to get m...


In [24]:
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [w for w in [lemmer.stem(token) for token in tokens] if not w in stop_words]

remove_punct_dict = dict((ord(punct), ' ') for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [25]:
df['story_processed'] = df.story.apply(lambda x : LemNormalize(x))
df['story_lines'] = df.story_processed.apply(lambda x: ' '.join(x))

/home/ifte-home/venvs/experiments/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/ifte-home/venvs/experiments/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
df.to_csv('processed_data.csv')

df = pd.read_csv('processed_data.csv')

df

,Unnamed: 0,storytitle,story,story_processed,story_lines
0,0,David Drops the Weight,David noticed he had put on a lot of weight re...,"['david', 'notic', 'put', 'lot', 'weight', 're...",david notic put lot weight recent examin hi ha...
1,1,Frustration,Tom had a very short temper.One day a guest ma...,"['tom', 'veri', 'short', 'temper', 'one', 'day...",tom veri short temper one day guest made veri ...
2,2,Marcus Buys Khakis,Marcus needed clothing for a business casual e...,"['marcu', 'need', 'cloth', 'busi', 'casual', '...",marcu need cloth busi casual event hi cloth ei...
3,3,Different Opinions,Bobby thought Bill should buy a trailer and ha...,"['bobbi', 'thought', 'bill', 'buy', 'trailer',...",bobbi thought bill buy trailer haul hi car bil...
4,4,Overcoming shortcomings,John was a pastor with a very bad memory.He tr...,"['john', 'wa', 'pastor', 'veri', 'bad', 'memor...",john wa pastor veri bad memori tri memor hi se...
...,...,...,...,...,...
52660,52660,Flavor,The man liked the flavor.He tried to recreate ...,"['man', 'like', 'flavor', 'tri', 'recreat', 'h...",man like flavor tri recreat home could get fla...
52661,52661,After Death,"After my friend's dad's funeral, I got in trou...","['friend', 'dad', 'funer', 'got', 'troubl', 'p...",friend dad funer got troubl princip said allow...
52662,52662,Janice breaks her wrist,Janice was out exercising for her big soccer g...,"['janic', 'wa', 'exercis', 'big', 'soccer', 'g...",janic wa exercis big soccer game wa drill leg ...
52663,52663,Jamie marries for love,Jamie is an american girl.Jamie wants to get m...,"['jami', 'american', 'girl', 'jami', 'want', '...",jami american girl jami want get marri mexican...


In [36]:
TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words=stop_words_nltk)

In [28]:
sent_tokens = df.story_lines.values.tolist()

In [38]:
tfidf = TfidfVec.fit_transform(sent_tokens)

In [45]:
def response(user_response):
    robo_response=''
    user_response = ' '.join(LemNormalize(user_response))
    user_response_tfidf = TfidfVec.fit_transform(list(user_response))
    vals = cosine_similarity(user_response_tfidf, tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [46]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [47]:
flag=True
print("ROBO: My name is Robo. Just Ask me anything. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    if user_response =='':
        continue
    else:
        user_response=user_response.lower()
        if(user_response!='bye'):
            if(user_response=='thanks' or user_response=='thank you' ):
                flag=False
                print("ROBO: You are welcome..")
            else:
                if(greeting(user_response)!=None):
                    print("ROBO: "+greeting(user_response))
                else:
                    print("ROBO: ",end="")
                    print(response(user_response))
                    sent_tokens.remove(user_response)
        else:
            flag=False
            print("ROBO: Bye! take care..")

ROBO: My name is Robo. Just Ask me anything. If you want to exit, type Bye!
jim
ROBO: 

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 1 while Y.shape[1] == 17292